Напишите декоратор, оптимизирующий работу декорируемой функции. Декоратор должен сохранять результат работы функции на ближайшие три запуска и вместо выполнения функции возвращать сохранённый результат. 
После трёх запусков функция должна вызываться вновь, а результат работы функции — вновь кешироваться.


In [16]:

from datetime import datetime
import time

def timeIt(func):
    time_list =[]
    def wrapped():
        nonlocal time_list
        if len(time_list)<1:
            for i in range(3):            
                time_list.append(func())
    
        return time_list.pop()
    return wrapped

@timeIt
def func_time():
    return datetime.now()

for i in range(7):
    print(func_time())
    time.sleep(1)





2024-11-21 10:38:07.707755
2024-11-21 10:38:07.707755
2024-11-21 10:38:07.707755
2024-11-21 10:38:10.709948
2024-11-21 10:38:10.709948
2024-11-21 10:38:10.709948
2024-11-21 10:38:13.712237


1. декорируем функцию, возвращаемую текущее время
2. Пишем декоратор, хранящий список кэшированных значений time_list. Если осталось меньше одного значения в кэше, запускаем кэширование 3 раза.
3. После запуска видим, что функция кэширует 3 значения за 1 запуск.

Задание 2. Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

1 создадим коллекцию url images:

In [55]:
images = []
import requests
from bs4 import BeautifulSoup
#list_url=["https://habr.com/ru/feed/"]
list_url=["https://bbc.com"]
for url in list_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    for img in soup.findAll('img'):
        images.append(img.get('src')) 

Находим количество картинок:

In [56]:
print(len(images))
images

80


['/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/8250/live/6b84c8d0-a7dd-11ef-a4fe-a3e9a6c5d640.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/2ba4/live/27dead50-a7c7-11ef-83ee-618e27a5f901.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/8250/live/6b84c8d0-a7dd-11ef-a4fe-a3e9a6c5d640.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/2ba4/live/27dead50-a7c7-11ef-83ee-618e27a5f901.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/ebb5/live/54e0cd20-a76c-11ef-a4fe-a3e9a6c5d640.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/images/ic/480x270/p0k59h0l.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/images/ic/480x270/p0k5m02k.jpg.webp',
 '/bbcx/grey-placeholder.png',
 'https://ichef.bbci.co.uk/news/480/cpsprodpb/c723/live/a043b2b0-a79d-11ef-bdf5-b7cb2fa86e10.jpg.webp'

Создадим функцию последовательной загрузки

In [57]:
import time
import urllib.request
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def download_images_consequent(image):    
    i=1    
    dir = "d:\CourcesML\ "
    filename = dir +  image.split('/')[-1] 
    try:
        urllib.request.urlretrieve(image,filename) 
        print("file downloaded")   
    except:
        print("An exception occurred") 
           


Создадим функцию загрузки файла из Url:

In [58]:
def download_file_by_url(url):
    # Генерируем имя файла, убирая недопустимые символы
    file_name = url.split('/')[-1]
    
    with requests.get(url, stream=True) as request:
        request.raise_for_status()
        with open(file_name, 'wb') as f:
            for piece in request.iter_content(chunk_size=8192):
                f.write(piece)
    return file_name

Создадим функцию, загружающую файлы в параллельных потоках

In [59]:
def download_files_in_threads(images):
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(download_file_by_url, images)
        #executor.map(download_images_consequent, images)

Создадим функцию, загружающую файлы в параллельных процессах:

In [60]:
def download_files_in_processes(images):
      with ProcessPoolExecutor(max_workers=4) as executor:
        executor.map(download_file_by_url, images)
        #executor.map(download_images_consequent, images)

In [61]:
 # Расчет времени последовательного метода
 
 
start_time = time.time()    
for url in images:
   download_images_consequent(url)
result_time = time.time() - start_time
print("Sequential download time: " ,result_time, "  seconds")

# Расчет времени загрузки использованием потоков

start_time = time.time()
download_files_in_threads(images)
thread_time = time.time() - start_time
print(" download time in threads: " ,thread_time, "  seconds")    

# Сравнение производительности с использованием процессов
    
start_time = time.time()
    
download_files_in_processes(images)    
process_time = time.time() - start_time
print(" download time in processes: " ,process_time, "  seconds")
    

An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception occurred
file downloaded
An exception

ВЫвод: Многопоточный вывод в разы быстрее параллельного, пробовал на разных сайтах. На процессах работает чуть быстрее. 
Почему происходят ошибки при загрузке файлов остается загадкой.